In [2]:
!pip install audiomentations tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2


In [3]:
import tensorflow as tf
import numpy as np
from argparse import ArgumentParser
from itertools import chain
import os
import tensorflow as tf
import librosa
from audiomentations import Compose, AddBackgroundNoise, TimeMask, Resample
from tqdm import tqdm
import tensorflow_addons as tfa


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [16]:
parser=ArgumentParser()
parser.add_argument("--data_dir",default="/kaggle/input/cw-attack-5000/data/")
parser.add_argument("--batch_size", default=128, type=int)
parser.add_argument("--lr", default=0.001, type=float)
parser.add_argument("--weight_decay", default=1e-4, type=float)
args = parser.parse_args([])
print(vars(args))

{'data_dir': '/kaggle/input/cw-attack-5000/data/', 'batch_size': 128, 'lr': 0.001, 'weight_decay': 0.0001}


In [5]:
def convert_to_mfccs(data, perm=None):
#     print(data.shape)
    stfts = tf.signal.stft(tf.squeeze(data), frame_length=480, frame_step=160,
                           fft_length=480)
    spectrograms = tf.abs(stfts)

    # Warp the linear scale spectrograms into the mel-scale.
    num_spectrogram_bins = stfts.shape[-1]
    lower_edge_hertz, upper_edge_hertz, num_mel_bins = 20.0, 7600.0, 80
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
      num_mel_bins, num_spectrogram_bins, 16000, lower_edge_hertz,
      upper_edge_hertz)
    mel_spectrograms = tf.tensordot(
      spectrograms, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
      linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    # Compute MFCCs from log_mel_spectrograms and take the first 13.
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(
      log_mel_spectrograms)[..., :40]
    
    if perm is not None:
        mfccs = tf.transpose(mfccs).numpy()
        mfccs = tf.convert_to_tensor(mfccs[perm])
        mfccs = tf.transpose(mfccs, perm=[2, 0, 1])
    else:
        mfccs = tf.transpose(mfccs, perm=[0, 2, 1])
    mfccs = tf.expand_dims(mfccs, axis=-1)
    return mfccs

In [6]:
def get_label(file_path):
    """
    Returns label of an audio file
    :param file_path:
    :return: Label
    """
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [7]:
def get_mfccs(file_path):
    """
    Takes an audio file and returns mfcc of the audio file. This will be used to generate
    the mfcc of validation audio files and test audio files
    :param file_path: audio file in wav format
    :return: mfcc
    """
    signal, sr = librosa.load(file_path, sr=16000)
    edited_signal = librosa.util.fix_length(signal, 16000)
#     log_spect = np.log(get_spectrogram(edited_signal) + 1e-6)

#     mfccs = librosa.feature.mfcc(y=edited_signal, n_mfcc=40, sr=sr, S=log_spect)
    return edited_signal

In [8]:
def create_example(fp,commands,data_dir,trainable=False):
    file_path=fp
    mfccs=get_mfccs(file_path)
    label = get_label(file_path)
    label = label.numpy().decode('utf-8')
    label_id = [i for i in range(len(commands)) if commands[i] == label][0]
#     if mfccs.shape != (40, 98):
#         print(mfccs.shape)
        # print("not padded")
    return np.expand_dims(mfccs, axis=-1), label_id
    

In [9]:
def load_files(data_dir='data'):
    """
    Loads the speech command data directory and returns the files and the labels
    :param data_dir:  path to directory containing audio files
    :return: filenames, commands
    """
    print(data_dir, '\n\n\n\n')
    commands = np.array(tf.io.gfile.listdir(str(data_dir)))
#     not_needed = ['README.md', 'speech_commands_v0.01.tar.gz', 'validation_list.txt', 'testing_list.txt', 'LICENSE',
#                   '.DS_Store', '_background_noise_']
    #commands = ['on', 'off', 'yes', 'no', 'stop', 'go', 'left', 'right', 'up', 'down']
    commands = ['on', 'off']
    filenames = []
    for i in range(len(commands)):
        filenames.append(tf.io.gfile.glob(str(data_dir) + "/" + commands[i] + "/*.wav"))
    filenames = list(chain.from_iterable(filenames))
    filenames = tf.random.shuffle(filenames)
    num_samples = len(filenames)
    print('Number of total examples:', num_samples)
    return filenames, commands

In [10]:
def load_test_files(filenames,data_dir):
    with open(os.path.join(data_dir, 'testing_list.txt'), 'r') as f:
        list_of_test_data = f.read().splitlines()
    test_files=[]
    for i in filenames:
        current_file = i.numpy().decode('utf-8')
        main_dir, label, data = current_file.rsplit('/', 2)
        input_data = '/'.join((label, data))
        if input_data in list_of_test_data:
            input_data = "/".join((data_dir, input_data))
            test_files.append(input_data)
    print("Completed loading test files")
    return test_files
            
        

In [11]:
def load_data(test_files,commands,data_dir):
    test_data=[]
    test_label=[]
    for fp in tqdm(test_files):

        data, label =  create_example(fp, commands, data_dir,trainable=False)
        test_data.append(data)
        test_label.append(label)

    test_data = np.array(test_data, dtype=np.float32)
    test_label = np.array(test_label, dtype=np.float32)

    print(f"Test data shape: {test_data.shape}, test label shape: {test_label.shape}")
    return test_data,test_label
    
    

In [12]:
def collect_data(batch_size,data_dir):
    filenames,commands=load_files(data_dir)
    test_files=load_test_files(filenames,data_dir)
    test_data,test_label=load_data(test_files,commands,data_dir)
    test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_label))
    return test_ds,commands
    
    
    
    

In [17]:
import time
start=time.time()
model = tf.keras.models.load_model('/kaggle/input/cw-model-5000/KWS_transformer',compile=False)
model.load_weights('/kaggle/input/cw-model-5000/best_weights')
np.random.seed(1234)
perm = np.random.permutation(40)
model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True
        ),
        optimizer=tfa.optimizers.AdamW(
            learning_rate=args.lr, weight_decay=args.weight_decay
        ),
        metrics=["accuracy"],
    )
test_ds,commands=collect_data(batch_size=args.batch_size,data_dir=args.data_dir)
test_ds=test_ds.batch(args.batch_size)
loss,acc,batches=0,0,0
for data,labels in test_ds:
    mfccs=convert_to_mfccs(data,perm=perm)
    l,a=model.test_on_batch(mfccs,labels)
    loss+=l
    acc+=a
    batches+=1
print(f"Test: loss:{loss/batches},acc:{acc/batches*100}")    
end=time.time()
print(end-start)

/kaggle/input/cw-attack-5000/data/ 




Number of total examples: 7718
Completed loading test files


  0%|          | 0/128 [00:00<?, ?it/s]/tmp/ipykernel_32/387395141.py:9: FutureWarning: Pass size=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  edited_signal = librosa.util.fix_length(signal, 16000)
100%|██████████| 128/128 [00:00<00:00, 131.55it/s]


Test data shape: (128, 16000, 1), test label shape: (128,)
Test: loss:3.6155033111572266,acc:34.375
29.454954147338867
